In [1]:
import json
import ast
import pandas as pd
import urllib.parse
import urllib.request
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
UK_Towns = pd.read_csv(r'.\Downloads\uk-towns\csv\uk-towns.csv')

In [6]:
UK_Towns.head()

,id,name,county,country,grid_reference,easting,northing,latitude,longitude,postcode_sector,local_government_area,nuts_region,type
0,1,Aaron's Hill,Surrey,England,SU957435,495783,143522,51.18291,-0.63098,GU7 2,Waverley District,South East,Suburban Area
1,2,Abbas Combe,Somerset,England,ST707226,370749,122688,51.00283,-2.41825,BA8 0,South Somerset District,South West,Village
2,3,Abberley,Worcestershire,England,SO744675,374477,267522,52.30522,-2.37574,WR6 6,Malvern Hills District,West Midlands,Village
3,4,Abberton,Essex,England,TM006190,600637,219093,51.83440,0.91066,CO5 7,Colchester District,Eastern,Village
4,5,Abberton,Worcestershire,England,SO995534,399538,253477,52.17955,-2.00817,WR10 2,Wychavon District,West Midlands,Hamlet


In [4]:
UK_Lat_Bounds = [49.86, 60.86]
UK_Long_Bounds = [-8.45, 1.78]

In [17]:
UniqueUserUKLocations = {}

with open(r'.\Downloads\locs.json') as f:
    for line in f:
        line_data = json.loads(line)
        UserID = line_data['userId']
        Coords = line_data['loc']['coordinates']
        
        if not UserID in UniqueUserUKLocations.keys():
            if (UK_Lat_Bounds[0] <= Coords[1] <= UK_Lat_Bounds[1]) & (UK_Long_Bounds[0] <= Coords[0] <= UK_Long_Bounds[1]):
                UniqueUserUKLocations[UserID] = Coords

In [19]:
UniqueUserUKLocations[list(UniqueUserUKLocations.keys())[0]]

[-1.481044469187128, 53.0016376403971]

In [20]:
long, lat = UniqueUserUKLocations[list(UniqueUserUKLocations.keys())[0]]

In [30]:
UK_Towns.loc[((UK_Towns['latitude'] - lat)**2 + (UK_Towns['longitude'] - long)**2).argmin()]

id                                       28451
name                                   Milford
county                              Derbyshire
country                                England
grid_reference                        SK351451
easting                                 435127
northing                                345116
latitude                               53.0022
longitude                             -1.47801
postcode_sector                         DE56 0
local_government_area    Amber Valley District
nuts_region                      East Midlands
type                                   Village
Name: 28450, dtype: object

In [35]:
UserLocs = pd.DataFrame(columns=['User ID', 'County', 'Local Gov Area', 'Country'])
i = 0

for UserID in UniqueUserUKLocations.keys():
    Long, Lat = UniqueUserUKLocations[UserID]
    LocData = UK_Towns.loc[((UK_Towns['latitude'] - Lat)**2 + (UK_Towns['longitude'] - Long)**2).argmin()]
    
    UserLocs.loc[i] = [UserID, LocData['county'], LocData['local_government_area'], LocData['country']]
    i += 1

In [41]:
UserLocs.head(10)

,User ID,County,Local Gov Area,Country
0,73888,Derbyshire,Amber Valley District,England
1,53511,Dorset,North Dorset District,England
2,81056,Lancashire,Burnley District,England
3,62156,West Midlands,Birmingham District,England
4,37916,Hertfordshire,Hertsmere District,England
5,61558,West Midlands,Solihull District,England
6,50934,Norfolk,King's Lynn and West Norfolk District,England
7,41938,Devon,East Devon District,England
8,50433,Greater Manchester,Stockport District,England
9,96607,Tyne and Wear,Newcastle upon Tyne District,England


In [49]:
UserLocs.groupby(['Country']).size().sort_values(ascending=False)

Country
England             32813
Scotland             2479
Wales                1404
Northern Ireland      751
Isle of Man            13
dtype: int64

In [50]:
UserLocs.groupby(['County']).size().sort_values(ascending=False)

County
Greater London                      6841
West Midlands                       1733
Greater Manchester                  1567
West Yorkshire                      1234
Hampshire                           1081
Essex                               1047
Kent                                1021
Hertfordshire                        852
Surrey                               738
Lancashire                           738
South Yorkshire                      720
Nottinghamshire                      709
Merseyside                           691
Leicestershire                       669
Tyne and Wear                        637
North Yorkshire                      634
Devon                                602
Buckinghamshire                      591
Berkshire                            579
Dorset                               561
Somerset                             553
Cheshire                             553
Derbyshire                           551
Staffordshire                        515
East Suss

In [51]:
UserLocs.groupby(['Local Gov Area']).size().sort_values(ascending=False)

Local Gov Area
Birmingham District                772
Leeds District                     476
Glasgow City                       388
Manchester District                358
Croydon                            352
Sheffield District                 341
Camden                             319
City of Leicester                  304
City of Edinburgh                  304
Tower Hamlets                      297
Bradford District                  294
Barnet                             294
Brent                              292
City of Bristol                    287
Newham                             282
City of Nottingham                 274
City of Westminster                271
Ealing                             267
Liverpool District                 262
Cornwall                           259
Wandsworth                         257
Wiltshire                          256
Bromley                            252
Enfield                            248
County Durham                      245
Southwark 